In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# 加载数据
data = pd.read_csv('../complete_data.csv')

# 处理缺失值（如果有）
data.fillna(0, inplace=True)

# 特征和标签
X = data[['Year', 'Bronze', 'Gold', 'Silver', 'Participants', 'Events', 'is_host', 'Total']]
y = data['Total']

# 标准化特征
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [5]:
# 选择用于聚类的特征
cluster_features = ['Bronze', 'Gold', 'Silver', 'Participants', 'Events']

# 标准化聚类特征
X_cluster = data[cluster_features]
X_cluster_scaled = scaler.fit_transform(X_cluster)

# 使用K-means进行聚类
kmeans = KMeans(n_clusters=3, random_state=42)
data['Cluster'] = kmeans.fit_predict(X_cluster_scaled)

# 查看聚类结果
print(data['Cluster'].value_counts())

Cluster
0    7012
2     635
1      75
Name: count, dtype: int64


In [6]:
# 将聚类标签转换为独热编码
encoder = OneHotEncoder(sparse=False)
cluster_encoded = encoder.fit_transform(data[['Cluster']])

# 添加独热编码后的聚类标签到特征集
X_scaled = np.hstack((X_scaled, cluster_encoded))

# 添加交互特征（例如：Participants * Cluster）
for i in range(cluster_encoded.shape[1]):
    X_scaled = np.hstack((X_scaled, X_scaled[:, 4].reshape(-1, 1) * cluster_encoded[:, i].reshape(-1, 1)))

# 添加东道主奖励项
X_scaled = np.hstack((X_scaled, X_scaled[:, 6].reshape(-1, 1) * 0.3))

TypeError: OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'